## Cherry Blossom Data

Aono et al. have collected a spectacular series of phenological data about the blossoming of *prunus jamasakura* in Kyoto, Japan.
For the cherry blossom data: https://ourworldindata.org/grapher/date-of-the-peak-cherry-tree-blossom-in-kyoto?tab=table
For meteorological data: https://www.data.jma.go.jp/stats/etrn/view/monthly_s3_en.php?block_no=47759&view=2

In this notebook we will analyse two questions:
 1. Since when and how much did the blossoming timing change?
 2. Is there any correlation to mean air temperature or other weather variables?
 
(cc) conrad.jackisch@tbt.tu-freiberg.de

In [ ]:
#!uv pip install numpy pandas lux plotly seaborn

In [20]:
#loading required libraries
import pandas as pd
import numpy as np
import lux

#import static plot libraries
import seaborn as sns
sns.set_style('whitegrid', {'grid.linestyle': u'--'})

#import interactive plot libraries
import plotly.express as px


In [ ]:
import plotly.figure_factory as ff
import plotly.io as pio
pio.renderers.default='iframe' #for windows use 'notebook' instead
import plotly.graph_objects as go

We have prepared the data from the original sources at the website http://atmenv.envi.osakafu-u.ac.jp/aono/kyophenotemp4/ (unfortunately no longer functional) and https://www.data.jma.go.jp/obd/stats/etrn/view/monthly_s3_en.php?block_no=47759&view=1 into one Excel file. 

Now, let's load the data and take a first look at it.

In [76]:
kyoto = pd.read_excel('flowering_prunus_japan.xlsx',sheet_name='Merged',index_col=0)
kyoto

Button(description='Toggle Pandas/Lux', layout=Layout(top='5px', width='140px'), style=ButtonStyle())

Output()

In [17]:
fig = px.scatter(kyoto, y='Flower')
#fig = px.scatter(kyoto, y='Flower', marginal_y='histogram')
#fig = px.scatter(kyoto, y='Flower', trendline='lowess', trendline_color_override='coral')
fig.update_layout({'template': 'none','title': 'Cherry Blossom Day of Year', 'xaxis_title': 'Year (AD)', 'yaxis_title': 'DOY'})
#fig.write_image('cherry_ts.pdf', width=1500, height=400)
fig.show()

In [18]:
fig = px.scatter(kyoto, y='MarTemp_any')
#fig = px.scatter(MarchT['Estimated temperature'],trendline='lowess',trendline_color_override='coral')
fig.update_layout({'template': 'none','title': 'Air Temperature in March, Kyoto', 'xaxis_title': 'Year (AD)', 'yaxis_title': 'Temp (°C)'})
fig.show()

In [27]:
#create new columns for decades to separate values later
kyoto['Decade'] = np.round(kyoto.index.values,-1)
kyoto['Dec25'] = (np.round(kyoto.index.values/2.5,-1)*2.5).astype(int)
kyoto['expTime'] = np.exp(kyoto.index.values/500.)

#boxplot with decades
fig = px.box(kyoto.loc[(kyoto.Decade>1600)], x="Decade", y="Flower")
#fig = px.box(kyoto, x="Dec25", y="Flower")
fig.update_layout({'template': 'none','title': 'Cherry Blossom Data'})
fig.show()

In [ ]:
# add a function for plotting the distributions

def plot_flower_distribution_shift(df, split_year=None, kde=True, bins=30, interactive=False):
    """
    Create plot showing distribution shift before/after split_year
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame with Year as index and 'Flower' column
    split_year : int, optional
        Year to split the data (required if interactive=False)
    kde : bool, default=True
        If True, use KDE plot; if False, use histogram
    bins : int, default=30
        Number of bins for histogram
    interactive : bool, default=False
        If True, create slider for split year (years 1800-2015)
        
    Returns:
    --------
    (fig, statx) : tuple
        Figure and statistics dictionary
    """
    
    if not interactive and split_year is None:
        raise ValueError("split_year must be provided when interactive=False")
    
    years = sorted(df.index.unique())
    
    # For interactive: only years between 1800 and 2015
    if interactive:
        split_years = [y for y in years if 1800 <= y <= 2015]
    else:
        split_years = [split_year]
    
    # Create subplots with boxplot on top
    fig = make_subplots(
        rows=2, cols=1,
        row_heights=[0.2, 0.8],
        vertical_spacing=0.08,
        subplot_titles=('', ''),
        specs=[[{"type": "box"}], [{"type": "xy"}]]
    )
    
    # Store annotations for each split year
    all_annotations = []
    
    # Create plots for each split year
    for i, year in enumerate(split_years):
        before = df.loc[df.index < year, 'Flower'].dropna()
        after = df.loc[df.index >= year, 'Flower'].dropna()
        
        if len(before) < 5 or len(after) < 5:
            continue
        
        visible = not interactive or i == 0
        
        # Add boxplots (row 1)
        fig.add_trace(go.Box(
            x=before,
            name=f'Before {year}',
            marker_color='steelblue',
            orientation='h',
            visible=visible,
            showlegend=False
        ), row=1, col=1)
        
        fig.add_trace(go.Box(
            x=after,
            name=f'From {year}',
            marker_color='coral',
            orientation='h',
            visible=visible,
            showlegend=False
        ), row=1, col=1)
        
        if kde:
            # KDE plots (row 2)
            kde_b = stats.gaussian_kde(before)
            kde_a = stats.gaussian_kde(after)
            x = np.linspace(min(before.min(), after.min()), 
                          max(before.max(), after.max()), 500)
            
            fig.add_trace(go.Scatter(
                x=x, y=kde_b(x), 
                name=f'Before {year} (n={len(before)})',
                line=dict(color='steelblue', width=2.5), 
                fill='tozeroy',
                fillcolor='rgba(70,130,180,0.2)', 
                visible=visible,
                showlegend=True
            ), row=2, col=1)
            
            fig.add_trace(go.Scatter(
                x=x, y=kde_a(x), 
                name=f'From {year} (n={len(after)})',
                line=dict(color='coral', width=2.5), 
                fill='tozeroy',
                fillcolor='rgba(255,127,80,0.2)', 
                visible=visible,
                showlegend=True
            ), row=2, col=1)
        else:
            # Histograms (row 2)
            fig.add_trace(go.Histogram(
                x=before, 
                name=f'Before {year} (n={len(before)})',
                marker_color='steelblue', 
                opacity=0.6, 
                nbinsx=bins,
                histnorm='probability density', 
                visible=visible,
                showlegend=True
            ), row=2, col=1)
            
            fig.add_trace(go.Histogram(
                x=after, 
                name=f'From {year} (n={len(after)})',
                marker_color='coral', 
                opacity=0.6, 
                nbinsx=bins,
                histnorm='probability density', 
                visible=visible,
                showlegend=True
            ), row=2, col=1)
        
        # Calculate statistics
        ks_stat, ks_p = stats.ks_2samp(before, after)
        
        # Mann-Whitney U test (more stringent for location shifts)
        mw_stat, mw_p = stats.mannwhitneyu(before, after, alternative='two-sided')
        
        # Cohen's d (effect size - more meaningful than p-value)
        cohens_d = (after.mean() - before.mean()) / np.sqrt(
            ((len(before)-1)*before.std()**2 + (len(after)-1)*after.std()**2) / 
            (len(before) + len(after) - 2)
        )
        
        # Determine significance level for Mann-Whitney
        if mw_p < 0.001:
            sig_level = "***"
        elif mw_p < 0.01:
            sig_level = "**"
        elif mw_p < 0.05:
            sig_level = "*"
        else:
            sig_level = "n.s."
        
        # Effect size interpretation
        if abs(cohens_d) < 0.2:
            effect = "negligible"
        elif abs(cohens_d) < 0.5:
            effect = "small"
        elif abs(cohens_d) < 0.8:
            effect = "medium"
        else:
            effect = "large"
        
        # Create summary statistics text
        stats_text = (
            f"<b>Before {year}</b><br>"
            f"μ = {before.mean():.1f} days<br>"
            f"σ = {before.std():.1f} days<br>"
            f"median = {before.median():.1f} days<br>"
            f"n = {len(before)}<br>"
            f"<br>"
            f"<b>From {year}</b><br>"
            f"μ = {after.mean():.1f} days<br>"
            f"σ = {after.std():.1f} days<br>"
            f"median = {after.median():.1f} days<br>"
            f"n = {len(after)}<br>"
            f"<br>"
            f"<b>Statistical Tests</b><br>"
            f"Δμ = {after.mean() - before.mean():.1f} days<br>"
            f"Cohen's d = {cohens_d:.2f} ({effect})<br>"
            f"Mann-Whitney U: p={mw_p:.4e} {sig_level}<br>"
            f"KS: D={ks_stat:.3f}, p={ks_p:.4e}"
        )
        
        # Store annotation for this year
        all_annotations.append({
            'text': stats_text,
            'xref': 'paper',
            'yref': 'paper',
            'x': 0.98,
            'y': 0.7,
            'xanchor': 'right',
            'yanchor': 'top',
            'showarrow': False,
            'align': 'left',
            'bgcolor': 'rgba(255, 255, 255, 0.8)',
            'bordercolor': 'black',
            'borderwidth': 1,
            'font': dict(size=10, family='monospace'),
            'visible': visible
        })
    
    # Layout
    fig.update_xaxes(title_text="Day of Year (Flowering)", row=1, col=1)
    fig.update_xaxes(title_text="Day of Year (Flowering)", row=2, col=1)
    fig.update_yaxes(title_text="", row=1, col=1)
    fig.update_yaxes(title_text="Probability Density", row=2, col=1)
    
    fig.update_layout(
        title='Flowering Time Distribution Shift',
        template='plotly_white',
        width=1100, 
        height=800,
        barmode='overlay',
        hovermode='x unified',
        legend=dict(
            yanchor="top",
            y=0.65,
            xanchor="left",
            x=0.02
        ),
        annotations=all_annotations
    )
    
    # Add slider if interactive
    if interactive:
        steps = []
        for i, year in enumerate(split_years):
            # Update annotations visibility for this step
            updated_annotations = []
            for j, ann in enumerate(all_annotations):
                ann_copy = ann.copy()
                ann_copy['visible'] = (j == i)
                updated_annotations.append(ann_copy)
            
            # Each year has 4 traces: 2 boxplots + 2 distributions
            step = dict(
                method="update",
                args=[
                    {"visible": [j in [i*4, i*4+1, i*4+2, i*4+3] for j in range(len(fig.data))]},
                    {"annotations": updated_annotations}
                ],
                label=str(year)
            )
            
            steps.append(step)
        
        fig.update_layout(
            sliders=[dict(
                active=0, 
                yanchor="top", 
                y=-0.08, 
                xanchor="left",
                currentvalue=dict(prefix="Split Year: "),
                pad=dict(t=50), 
                steps=steps
            )]
        )
    
    # Calculate stats for the current/first split year (for output)
    current_year = split_years[0] if interactive else split_year
    before = df.loc[df.index < current_year, 'Flower'].dropna()
    after = df.loc[df.index >= current_year, 'Flower'].dropna()
    ks_stat, ks_p = stats.ks_2samp(before, after)
    mw_stat, mw_p = stats.mannwhitneyu(before, after, alternative='two-sided')
    cohens_d = (after.mean() - before.mean()) / np.sqrt(
        ((len(before)-1)*before.std()**2 + (len(after)-1)*after.std()**2) / 
        (len(before) + len(after) - 2)
    )
    
    statx = {
        'split_year': current_year,
        'mean_before': before.mean(), 
        'mean_after': after.mean(),
        'mean_difference': after.mean() - before.mean(),
        'std_before': before.std(), 
        'std_after': after.std(),
        'median_before': before.median(),
        'median_after': after.median(),
        'cohens_d': cohens_d,
        'mannwhitney_u': mw_stat,
        'mannwhitney_p': mw_p,
        'ks_statistic': ks_stat, 
        'ks_p_value': ks_p
    }
    
    return fig, statx

In [ ]:
fig, statx = plot_flower_distribution_shift(kyoto, split_year=1950, kde=True)
#fig, statx = plot_flower_distribution_shift(kyoto, kde=True, interactive=True) #this calls the interactive version
fig.show()

In [93]:
px.scatter(kyoto, x='T_JFM', y='Flower', hover_name=kyoto.index, trendline='ols', template='none')
#px.scatter(kyoto, x='Prec_JFM', y='Flower', hover_name=kyoto.index, trendline='ols', template='none')
#px.scatter(kyoto, x='Sun_JFM', y='Flower', hover_name=kyoto.index, trendline='ols', template='none')